In [3]:
#Import Dependencies
import pandas as pd
import os
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from pprint import pprint
import json
import csv, sqlite3
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func,inspect, desc, and_, or_

print(os.getcwd())

/Users/vrishal/Desktop/Combined Project 2


In [4]:
#Read CSV file to convert into dataframe
path = "Resources/data/select_top10.csv"
csvdf = pd.read_csv(path)
#csvdf.columns
#csvdf.loc[csvdf['IATA_Code_Operating_Airline']== 'EV','DOT_ID_Operating_Airline']

In [5]:
#Filter dataframe with selected 10 airports and 10 airlines
airports=["ATL", "BOS", "DAL", "DEN", "FLL", "JFK", "LAS", "LAX", "PDX", "SFO"]
airlines=["DL", "WN","UA","B6","AS","AA","OO","F9","EV","HA"]
ontime_data = csvdf[csvdf.Origin.isin(airports) & csvdf.Dest.isin(airports) & csvdf.IATA_Code_Operating_Airline.isin(airlines)]
ontime_data.head()

,Month,FlightDate,DOT_ID_Operating_Airline,IATA_Code_Operating_Airline,Flight_Number_Operating_Airline,OriginAirportID,Origin,DestAirportID,Dest,ArrDelayMinutes,Cancelled,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
3,1,2018-01-27,19977,UA,361,12889,LAS,14771,SFO,0.0,0.0,NaN,NaN,NaN,NaN,NaN
5,1,2018-01-27,19977,UA,358,14771,SFO,12889,LAS,0.0,0.0,NaN,NaN,NaN,NaN,NaN
11,1,2018-01-27,19977,UA,339,10721,BOS,11292,DEN,0.0,0.0,NaN,NaN,NaN,NaN,NaN
19,1,2018-01-27,19977,UA,325,12892,LAX,12889,LAS,0.0,0.0,NaN,NaN,NaN,NaN,NaN
20,1,2018-01-27,19977,UA,324,12889,LAS,12892,LAX,0.0,0.0,NaN,NaN,NaN,NaN,NaN


In [6]:
# Clean data by dropping NaN data 
for column in ['Month', 'DOT_ID_Operating_Airline', 'Flight_Number_Operating_Airline',
               'OriginAirportID', 'DestAirportID', 'ArrDelayMinutes']:
    pd.to_numeric(ontime_data[column], errors='coerce')
    ontime_data = ontime_data.dropna(subset=[column])
ontime_data.head()

,Month,FlightDate,DOT_ID_Operating_Airline,IATA_Code_Operating_Airline,Flight_Number_Operating_Airline,OriginAirportID,Origin,DestAirportID,Dest,ArrDelayMinutes,Cancelled,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
3,1,2018-01-27,19977,UA,361,12889,LAS,14771,SFO,0.0,0.0,NaN,NaN,NaN,NaN,NaN
5,1,2018-01-27,19977,UA,358,14771,SFO,12889,LAS,0.0,0.0,NaN,NaN,NaN,NaN,NaN
11,1,2018-01-27,19977,UA,339,10721,BOS,11292,DEN,0.0,0.0,NaN,NaN,NaN,NaN,NaN
19,1,2018-01-27,19977,UA,325,12892,LAX,12889,LAS,0.0,0.0,NaN,NaN,NaN,NaN,NaN
20,1,2018-01-27,19977,UA,324,12889,LAS,12892,LAX,0.0,0.0,NaN,NaN,NaN,NaN,NaN


In [7]:
#ontime_data[ontime_data['ArrDelayMinutes'] == 0].count()
#Map created to convert the Airport code to full name
ontime_data['Airline'] = ontime_data['IATA_Code_Operating_Airline'].map({"DL": "Delta", "WN": "SouthWest","UA": "United Airlines",
                                                                         "B6":"Jet Blue","AS": "Alaska Airlines","AA": "American Airlines",
                                                                         "OO": "SkyWest","F9": "Frontier Airlines","EV": "Express JetAir",
                                                                         "HA": "Hawaiian Airlines" })
                                                                         
                                                                         
ontime_data.tail()                                                               

,Month,FlightDate,DOT_ID_Operating_Airline,IATA_Code_Operating_Airline,Flight_Number_Operating_Airline,OriginAirportID,Origin,DestAirportID,Dest,ArrDelayMinutes,Cancelled,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay,Airline
3170183,12,2018-12-18,20304,OO,3497,11259,DAL,14771,SFO,0.0,0.0,NaN,NaN,NaN,NaN,NaN,SkyWest
3170184,12,2018-12-18,20304,OO,3501,12889,LAS,12892,LAX,0.0,0.0,NaN,NaN,NaN,NaN,NaN,SkyWest
3170185,12,2018-12-18,20304,OO,3503,12889,LAS,12892,LAX,0.0,0.0,NaN,NaN,NaN,NaN,NaN,SkyWest
3170245,12,2018-12-18,20304,OO,3729,12478,JFK,10721,BOS,0.0,0.0,NaN,NaN,NaN,NaN,NaN,SkyWest
3170247,12,2018-12-18,20304,OO,3752,10721,BOS,12478,JFK,0.0,0.0,NaN,NaN,NaN,NaN,NaN,SkyWest


In [8]:
#Data munging done for the line chart visualization. Unable to finish it by submission time.
# from collections import defaultdict
# select_df = ontime_data.loc[(ontime_data['OriginAirportID'] == 10397) &
#                                 (ontime_data['DestAirportID'] == 10721)&
#                            (ontime_data['ArrDelayMinutes'] > 0)].groupby(['Airline', 'Month'])['Origin'].value_counts()

# result = defaultdict(list)
# for key, value in select_df.to_dict().items():
#     if key[0] not in result:
#         for month in range(12):
#             result[key[0]].append(0)
#     result[key[0]][key[1]-1] = value
    
# print(result)


In [9]:
#Push the filtered data into the database

engine = create_engine("sqlite:///Resources/data/ontime.sqlite")
Base = automap_base()
Base.metadata.create_all(engine)
Base.prepare(engine, reflect = True)

session = Session(bind=engine)    
ontime_data.to_sql(name='ontime', con=engine, if_exists='replace')

In [10]:
#Tested the month, airline and airport from code to user friendly values
month_conv = {
    "Jan": 1, "Feb": 2, "Mar": 3, "Apr": 4, "May": 5, "Jun": 6, "Jul": 7, "Aug": 8, "Sep": 9, "Oct": 10, "Nov": 11, "Dec": 12
}
airline_conv = {}
airline_groups = ontime_data.groupby(
    ['IATA_Code_Operating_Airline', 'DOT_ID_Operating_Airline']
)['IATA_Code_Operating_Airline'].unique().to_dict().keys()
airline_groups
for item in list(airline_groups):
    airline_conv[item[0]] = item[1]

airport_conv = {}
airport_groups = ontime_data.groupby(['Origin', 'OriginAirportID'])['Origin'].unique().to_dict().keys()
for item in list(airport_groups):
    airport_conv[item[0]] = item[1]
airline_conv['DL']

month = "Mar"
origin = "LAS"
dest = "LAX"

invalues = []
for airline in airlines:
    if airline in airline_conv:
        #print(airline, airline_conv[airline])
        invalues.append([month_conv[month], airline_conv[airline], airport_conv[origin], airport_conv[dest]])
invalues


[[3, 19790, 12889, 12892],
 [3, 19393, 12889, 12892],
 [3, 19977, 12889, 12892],
 [3, 20409, 12889, 12892],
 [3, 19930, 12889, 12892],
 [3, 19805, 12889, 12892],
 [3, 20304, 12889, 12892],
 [3, 20436, 12889, 12892]]

In [11]:
# Tested data for rendering airline arrival delay metrics to create a pie chart
selected_airline = invalues[2] # Assume user chose second airline ranked by delay prediction

select_df = ontime_data.loc[(ontime_data['Month'] == selected_airline[0]) &
                            (ontime_data['DOT_ID_Operating_Airline'] == selected_airline[1]) &
                             (ontime_data['OriginAirportID'] == selected_airline[2]) &
                             (ontime_data['DestAirportID'] == selected_airline[3])]

cd = int(select_df.loc[select_df['CarrierDelay'] > 0].count()['Month'])
wd = int(select_df.loc[select_df['WeatherDelay'] > 0].count()['Month'])
nd = int(select_df.loc[select_df['NASDelay'] > 0].count()['Month'])
sd = int(select_df.loc[select_df['SecurityDelay'] > 0].count()['Month'])
ld = int(select_df.loc[select_df['LateAircraftDelay'] > 0].count()['Month'])

print(cd, wd, nd, sd, ld)

4 0 5 0 4


In [12]:
# The prediction data
y = ontime_data["ArrDelayMinutes"]  

In [13]:
# Features selected to help predict the arrival delay minutes data using Decision Tree Regressor
ontime_features = ["Month","DOT_ID_Operating_Airline", "OriginAirportID", "DestAirportID"]
X = ontime_data[ontime_features]  


In [14]:
#Confirming whether X and Y has equal rows 
X.shape

(348614, 4)

In [15]:
#Confirming whether X and Y has equal rows 
y.shape

(348614,)

In [16]:
#Split data for model training
train_X, val_X, train_y, val_y = train_test_split(X, y, random_state=1)


In [17]:
#Used DecisionTreeRegressor model for prediction analysis
ontimemodel = DecisionTreeRegressor(random_state=1)

In [18]:
# training the model
ontimemodel.fit(train_X,train_y) 

DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=1, splitter='best')

In [19]:
#Tested the trained model on the split data
print("Making predictions for the following 8 airlines:")
print(invalues)
print("The predictions are")
val_predictions = ontimemodel.predict(val_X)
print(val_predictions)


Making predictions for the following 8 airlines:
[[3, 19790, 12889, 12892], [3, 19393, 12889, 12892], [3, 19977, 12889, 12892], [3, 20409, 12889, 12892], [3, 19930, 12889, 12892], [3, 19805, 12889, 12892], [3, 20304, 12889, 12892], [3, 20436, 12889, 12892]]
The predictions are
[19.73125    16.85185185  1.9375     ... 31.43956044  4.31818182
  3.26368159]


In [20]:
#Calculating and printing the mean absolute error
print(mean_absolute_error(val_y, val_predictions))

18.399300713260907


In [21]:
#Tested the trained model on user selected values from the dataset
val_predictions = []
for invalue in invalues:
    val_predictions.append(ontimemodel.predict([invalue])[0])
print(val_predictions)

[6.071428571428571, 20.322580645161292, 7.352941176470588, 8.08695652173913, 4.411764705882353, 9.818181818181818, 25.52112676056338, 17.0]


In [22]:
#Prediction analysis compared with the average of the Arrival Minutes Delay
val_means = []
index = 0
result_df = pd.DataFrame(columns=['Airline', 'Predicted Arrival Delay', 'Average Arrival Delay'])
for invalue in invalues:
    airline_df = ontime_data.loc[(ontime_data['Month'] == invalue[0]) &
                                 (ontime_data['DOT_ID_Operating_Airline'] == invalue[1]) &
                                 (ontime_data['OriginAirportID'] == invalue[2]) &
                                 (ontime_data['DestAirportID'] == invalue[3])]
    if airline_df.shape[0] == 0:
       # print("Airline: {}, Delay prediction not available".format(airlines[index]))
        result_df.loc[index] = [airlines[index], "Not Available", "Not Available"]
    else:
        predictedDelay = ontimemodel.predict([invalue])[0]
        averageDelay = airline_df['ArrDelayMinutes'].mean(axis=0)
        #print("Airline: {}, Predicted delay: {} minutes, Average delay: {} minutes".format(
            #airlines[index], predictedDelay, averageDelay))
        result_df.loc[index] = [airlines[index], predictedDelay, averageDelay]
    index = index + 1
result_df.sort_values(by=['Predicted Arrival Delay','Average Arrival Delay'], ascending=True)
#result_df

,Airline,Predicted Arrival Delay,Average Arrival Delay
0,DL,6.07143,7.35088
2,UA,7.35294,6.86567
5,AA,9.81818,9.42342
1,WN,20.3226,18.7238
6,OO,25.5211,22.6132
3,B6,Not Available,Not Available
4,AS,Not Available,Not Available
7,F9,Not Available,Not Available
